In [1]:
!pip install requests beautifulsoup4 numpy scikit-learn

**Punto 1**

In [5]:
import json
import datetime
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
from enum import Enum
import re
import openai  # Para integración con LLMs (opcional)

class TaskStatus(Enum):
    PENDING = "pendiente"
    IN_PROGRESS = "en_progreso"
    COMPLETED = "completada"
    OVERDUE = "vencida"

class TaskPriority(Enum):
    LOW = 1
    MEDIUM = 2
    HIGH = 3
    URGENT = 4

@dataclass
class Task:
    id: str
    title: str
    description: str
    chapter: int
    exercise_number: int
    due_date: datetime.datetime
    priority: TaskPriority
    status: TaskStatus
    tags: List[str]
    requirements: List[str]
    estimated_hours: float
    progress_notes: List[str]
    created_at: datetime.datetime

    def to_dict(self):
        return {
            'id': self.id,
            'title': self.title,
            'description': self.description,
            'chapter': self.chapter,
            'exercise_number': self.exercise_number,
            'due_date': self.due_date.isoformat(),
            'priority': self.priority.value,
            'status': self.status.value,
            'tags': self.tags,
            'requirements': self.requirements,
            'estimated_hours': self.estimated_hours,
            'progress_notes': self.progress_notes,
            'created_at': self.created_at.isoformat()
        }

class CourseTaskAgent:
    def __init__(self, data_file: str = "course_tasks.json"):
        self.data_file = data_file
        self.tasks: Dict[str, Task] = {}
        self.load_tasks()
        self.initialize_course_tasks()

    def load_tasks(self):
        """Carga las tareas desde archivo JSON"""
        try:
            with open(self.data_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                for task_id, task_data in data.items():
                    # Convertir strings de fecha de vuelta a datetime
                    task_data['due_date'] = datetime.datetime.fromisoformat(task_data['due_date'])
                    task_data['created_at'] = datetime.datetime.fromisoformat(task_data['created_at'])
                    task_data['priority'] = TaskPriority(task_data['priority'])
                    task_data['status'] = TaskStatus(task_data['status'])

                    self.tasks[task_id] = Task(**task_data)
        except FileNotFoundError:
            print("No se encontró archivo de tareas. Creando uno nuevo...")

    def save_tasks(self):
        """Guarda las tareas en archivo JSON"""
        data = {task_id: task.to_dict() for task_id, task in self.tasks.items()}
        with open(self.data_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

    def initialize_course_tasks(self):
        """Inicializa las tareas del curso basadas en el documento"""
        if not self.tasks:  # Solo si no hay tareas cargadas
            course_tasks_data = [
                {
                    "title": "Definiciones de IA con LLMs",
                    "description": "Pedir definiciones de IA a varios LLMs y crear definición propia",
                    "chapter": 1,
                    "exercise": 1,
                    "tags": ["definiciones", "LLMs", "conceptos"],
                    "requirements": ["Acceso a múltiples LLMs", "Análisis comparativo"],
                    "hours": 2
                },
                {
                    "title": "Análisis de video sobre IA",
                    "description": "Ver video en YouTube y hacer análisis comparativo",
                    "chapter": 1,
                    "exercise": 2,
                    "tags": ["video", "análisis", "comparación"],
                    "requirements": ["Acceso a YouTube", "Capacidad de análisis"],
                    "hours": 1.5
                },
                {
                    "title": "Autómatas Celulares 1D",
                    "description": "Maximizar función f(x)=x sen(10πx) + 1, con x ∈[0,1]",
                    "chapter": 3,
                    "exercise": 1,
                    "tags": ["optimización", "algoritmos genéticos", "matemáticas"],
                    "requirements": ["Python", "NumPy", "Matplotlib"],
                    "hours": 4
                },
                {
                    "title": "Sistema de democracia con AGs",
                    "description": "Distribuir poder político usando algoritmos genéticos",
                    "chapter": 3,
                    "exercise": 2,
                    "tags": ["algoritmos genéticos", "optimización", "simulación"],
                    "requirements": ["Python", "Algoritmos genéticos", "Matrices"],
                    "hours": 6
                },
                {
                    "title": "Distribución de energía eléctrica",
                    "description": "Optimizar despacho de energía minimizando costos",
                    "chapter": 3,
                    "exercise": 3,
                    "tags": ["optimización", "algoritmos genéticos", "energía"],
                    "requirements": ["Python", "Algoritmos genéticos", "Programación lineal"],
                    "hours": 8
                },
                {
                    "title": "Programación Genética - Circuito 7 segmentos",
                    "description": "Usar PG para diseñar circuito lógico codificador",
                    "chapter": 4,
                    "exercise": 2,
                    "tags": ["programación genética", "circuitos", "lógica"],
                    "requirements": ["Python", "DEAP o similar", "Conocimientos de lógica digital"],
                    "hours": 10
                },
                {
                    "title": "Clasificación Fashion MNIST",
                    "description": "Clasificar prendas de vestir usando TensorFlow",
                    "chapter": 5,
                    "exercise": 2,
                    "tags": ["redes neuronales", "tensorflow", "clasificación"],
                    "requirements": ["TensorFlow", "Python", "Jupyter"],
                    "hours": 6
                },
                {
                    "title": "Sistema RAG Universidad Nacional",
                    "description": "Desarrollar sistema RAG para información universitaria",
                    "chapter": 7,
                    "exercise": 3,
                    "tags": ["RAG", "chatbot", "información"],
                    "requirements": ["Python", "LangChain", "Vector DB", "LLM API"],
                    "hours": 12
                }
            ]

            for i, task_data in enumerate(course_tasks_data):
                task_id = f"task_{i+1:03d}"
                due_date = datetime.datetime.now() + datetime.timedelta(days=7*(i+1))

                task = Task(
                    id=task_id,
                    title=task_data["title"],
                    description=task_data["description"],
                    chapter=task_data["chapter"],
                    exercise_number=task_data["exercise"],
                    due_date=due_date,
                    priority=TaskPriority.MEDIUM,
                    status=TaskStatus.PENDING,
                    tags=task_data["tags"],
                    requirements=task_data["requirements"],
                    estimated_hours=task_data["hours"],
                    progress_notes=[],
                    created_at=datetime.datetime.now()
                )

                self.tasks[task_id] = task

            self.save_tasks()

    def add_task(self, title: str, description: str, chapter: int,
                 exercise_number: int, due_date: datetime.datetime,
                 priority: TaskPriority = TaskPriority.MEDIUM,
                 tags: List[str] = None, requirements: List[str] = None,
                 estimated_hours: float = 1.0) -> str:
        """Añade una nueva tarea"""
        task_id = f"task_{len(self.tasks)+1:03d}"

        task = Task(
            id=task_id,
            title=title,
            description=description,
            chapter=chapter,
            exercise_number=exercise_number,
            due_date=due_date,
            priority=priority,
            status=TaskStatus.PENDING,
            tags=tags or [],
            requirements=requirements or [],
            estimated_hours=estimated_hours,
            progress_notes=[],
            created_at=datetime.datetime.now()
        )

        self.tasks[task_id] = task
        self.save_tasks()
        return task_id

    def update_task_status(self, task_id: str, status: TaskStatus, note: str = ""):
        """Actualiza el estado de una tarea"""
        if task_id in self.tasks:
            self.tasks[task_id].status = status
            if note:
                self.tasks[task_id].progress_notes.append(
                    f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}: {note}"
                )
            self.save_tasks()
            return True
        return False

    def get_pending_tasks(self) -> List[Task]:
        """Obtiene tareas pendientes ordenadas por prioridad y fecha límite"""
        pending = [task for task in self.tasks.values()
                  if task.status == TaskStatus.PENDING]
        return sorted(pending, key=lambda x: (x.priority.value, x.due_date), reverse=True)

    def get_overdue_tasks(self) -> List[Task]:
        """Obtiene tareas vencidas"""
        now = datetime.datetime.now()
        overdue = []
        for task in self.tasks.values():
            if task.due_date < now and task.status != TaskStatus.COMPLETED:
                task.status = TaskStatus.OVERDUE
                overdue.append(task)

        if overdue:
            self.save_tasks()
        return overdue

    def get_tasks_by_chapter(self, chapter: int) -> List[Task]:
        """Obtiene tareas por capítulo"""
        return [task for task in self.tasks.values() if task.chapter == chapter]

    def search_tasks(self, query: str) -> List[Task]:
        """Busca tareas por título, descripción o tags"""
        query = query.lower()
        results = []

        for task in self.tasks.values():
            if (query in task.title.lower() or
                query in task.description.lower() or
                any(query in tag.lower() for tag in task.tags)):
                results.append(task)

        return results

    def get_progress_summary(self) -> Dict[str, Any]:
        """Obtiene resumen del progreso"""
        total_tasks = len(self.tasks)
        completed = len([t for t in self.tasks.values() if t.status == TaskStatus.COMPLETED])
        in_progress = len([t for t in self.tasks.values() if t.status == TaskStatus.IN_PROGRESS])
        pending = len([t for t in self.tasks.values() if t.status == TaskStatus.PENDING])
        overdue = len(self.get_overdue_tasks())

        total_hours = sum(task.estimated_hours for task in self.tasks.values())
        completed_hours = sum(task.estimated_hours for task in self.tasks.values()
                            if task.status == TaskStatus.COMPLETED)

        return {
            "total_tasks": total_tasks,
            "completed": completed,
            "in_progress": in_progress,
            "pending": pending,
            "overdue": overdue,
            "completion_percentage": (completed / total_tasks * 100) if total_tasks > 0 else 0,
            "total_estimated_hours": total_hours,
            "completed_hours": completed_hours,
            "remaining_hours": total_hours - completed_hours
        }

    def suggest_next_task(self) -> Optional[Task]:
        """Sugiere la próxima tarea a trabajar basada en prioridad y fechas"""
        pending_tasks = self.get_pending_tasks()
        overdue_tasks = self.get_overdue_tasks()

        if overdue_tasks:
            return max(overdue_tasks, key=lambda x: x.priority.value)
        elif pending_tasks:
            return pending_tasks[0]
        return None

    def generate_study_plan(self, days: int = 30) -> List[Dict[str, Any]]:
        """Genera un plan de estudio para los próximos días"""
        pending_tasks = self.get_pending_tasks()
        total_hours = sum(task.estimated_hours for task in pending_tasks)
        hours_per_day = total_hours / days if days > 0 else total_hours

        plan = []
        current_day = 0
        current_hours = 0
        daily_tasks = []

        for task in pending_tasks:
            if current_hours + task.estimated_hours > hours_per_day and daily_tasks:
                plan.append({
                    "day": current_day + 1,
                    "date": (datetime.datetime.now() + datetime.timedelta(days=current_day)).strftime('%Y-%m-%d'),
                    "tasks": daily_tasks.copy(),
                    "total_hours": current_hours
                })
                current_day += 1
                current_hours = 0
                daily_tasks = []

            daily_tasks.append({
                "id": task.id,
                "title": task.title,
                "hours": task.estimated_hours,
                "priority": task.priority.name
            })
            current_hours += task.estimated_hours

        if daily_tasks:
            plan.append({
                "day": current_day + 1,
                "date": (datetime.datetime.now() + datetime.timedelta(days=current_day)).strftime('%Y-%m-%d'),
                "tasks": daily_tasks,
                "total_hours": current_hours
            })

        return plan

    def interactive_menu(self):
        """Menú interactivo del agente"""
        while True:
            print("\n" + "="*50)
            print("🤖 AGENTE IA - GESTIÓN DE TAREAS DEL CURSO")
            print("="*50)
            print("1. Ver resumen de progreso")
            print("2. Ver todas las tareas")
            print("3. Ver tareas pendientes")
            print("4. Ver tareas vencidas")
            print("5. Buscar tareas")
            print("6. Actualizar estado de tarea")
            print("7. Sugerir próxima tarea")
            print("8. Generar plan de estudio")
            print("9. Ver tareas por capítulo")
            print("0. Salir")
            print("-"*50)

            choice = input("Seleccione una opción: ").strip()

            if choice == "1":
                self.show_progress_summary()
            elif choice == "2":
                self.show_all_tasks()
            elif choice == "3":
                self.show_pending_tasks()
            elif choice == "4":
                self.show_overdue_tasks()
            elif choice == "5":
                self.search_tasks_interactive()
            elif choice == "6":
                self.update_task_interactive()
            elif choice == "7":
                self.suggest_next_task_interactive()
            elif choice == "8":
                self.show_study_plan()
            elif choice == "9":
                self.show_tasks_by_chapter_interactive()
            elif choice == "0":
                print("¡Hasta luego! 🤖")
                break
            else:
                print("Opción no válida. Intente de nuevo.")

            input("\nPresione Enter para continuar...")

    def show_progress_summary(self):
        """Muestra resumen del progreso"""
        summary = self.get_progress_summary()
        print(f"\n📊 RESUMEN DE PROGRESO")
        print(f"Total de tareas: {summary['total_tasks']}")
        print(f"Completadas: {summary['completed']} ✅")
        print(f"En progreso: {summary['in_progress']} 🔄")
        print(f"Pendientes: {summary['pending']} ⏳")
        print(f"Vencidas: {summary['overdue']} ⚠️")
        print(f"Progreso: {summary['completion_percentage']:.1f}%")
        print(f"Horas estimadas totales: {summary['total_estimated_hours']}")
        print(f"Horas completadas: {summary['completed_hours']}")
        print(f"Horas restantes: {summary['remaining_hours']}")

    def show_task_details(self, task: Task):
        """Muestra detalles de una tarea"""
        status_icons = {
            TaskStatus.PENDING: "⏳",
            TaskStatus.IN_PROGRESS: "🔄",
            TaskStatus.COMPLETED: "✅",
            TaskStatus.OVERDUE: "⚠️"
        }

        priority_icons = {
            TaskPriority.LOW: "🟢",
            TaskPriority.MEDIUM: "🟡",
            TaskPriority.HIGH: "🟠",
            TaskPriority.URGENT: "🔴"
        }

        print(f"\n📋 {task.title}")
        print(f"   ID: {task.id}")
        print(f"   Capítulo: {task.chapter} - Ejercicio: {task.exercise_number}")
        print(f"   Estado: {status_icons[task.status]} {task.status.value}")
        print(f"   Prioridad: {priority_icons[task.priority]} {task.priority.name}")
        print(f"   Fecha límite: {task.due_date.strftime('%Y-%m-%d %H:%M')}")
        print(f"   Horas estimadas: {task.estimated_hours}")
        print(f"   Descripción: {task.description}")
        if task.tags:
            print(f"   Tags: {', '.join(task.tags)}")
        if task.requirements:
            print(f"   Requisitos: {', '.join(task.requirements)}")
        if task.progress_notes:
            print(f"   Notas de progreso:")
            for note in task.progress_notes[-3:]:  # Últimas 3 notas
                print(f"     - {note}")

    def show_all_tasks(self):
        """Muestra todas las tareas"""
        print(f"\n📝 TODAS LAS TAREAS ({len(self.tasks)})")
        for task in self.tasks.values():
            self.show_task_details(task)

    def show_pending_tasks(self):
        """Muestra tareas pendientes"""
        pending = self.get_pending_tasks()
        print(f"\n⏳ TAREAS PENDIENTES ({len(pending)})")
        for task in pending:
            self.show_task_details(task)

    def show_overdue_tasks(self):
        """Muestra tareas vencidas"""
        overdue = self.get_overdue_tasks()
        print(f"\n⚠️  TAREAS VENCIDAS ({len(overdue)})")
        for task in overdue:
            self.show_task_details(task)

    def search_tasks_interactive(self):
        """Búsqueda interactiva de tareas"""
        query = input("Ingrese término de búsqueda: ").strip()
        if query:
            results = self.search_tasks(query)
            print(f"\n🔍 RESULTADOS DE BÚSQUEDA para '{query}' ({len(results)})")
            for task in results:
                self.show_task_details(task)
        else:
            print("Debe ingresar un término de búsqueda.")

    def update_task_interactive(self):
        """Actualización interactiva de tareas"""
        task_id = input("Ingrese ID de la tarea: ").strip()
        if task_id in self.tasks:
            print("\nEstados disponibles:")
            for i, status in enumerate(TaskStatus, 1):
                print(f"{i}. {status.value}")

            try:
                status_choice = int(input("Seleccione nuevo estado: ")) - 1
                statuses = list(TaskStatus)
                if 0 <= status_choice < len(statuses):
                    new_status = statuses[status_choice]
                    note = input("Ingrese nota (opcional): ").strip()

                    self.update_task_status(task_id, new_status, note)
                    print(f"✅ Tarea {task_id} actualizada a {new_status.value}")
                else:
                    print("Opción no válida.")
            except ValueError:
                print("Debe ingresar un número válido.")
        else:
            print("ID de tarea no encontrado.")

    def suggest_next_task_interactive(self):
        """Sugerencia interactiva de próxima tarea"""
        next_task = self.suggest_next_task()
        if next_task:
            print("\n🎯 TAREA SUGERIDA:")
            self.show_task_details(next_task)
        else:
            print("\n🎉 ¡No hay tareas pendientes! Has completado todo.")

    def show_study_plan(self):
        """Muestra plan de estudio"""
        days = input("¿Para cuántos días desea el plan? (por defecto 30): ").strip()
        try:
            days = int(days) if days else 30
        except ValueError:
            days = 30

        plan = self.generate_study_plan(days)
        print(f"\n📅 PLAN DE ESTUDIO PARA {days} DÍAS")

        for day_plan in plan:
            print(f"\n📆 Día {day_plan['day']} ({day_plan['date']}) - {day_plan['total_hours']:.1f} horas")
            for task in day_plan['tasks']:
                print(f"   • {task['title']} ({task['hours']}h) - {task['priority']}")

    def show_tasks_by_chapter_interactive(self):
        """Muestra tareas por capítulo"""
        try:
            chapter = int(input("Ingrese número de capítulo: ").strip())
            tasks = self.get_tasks_by_chapter(chapter)
            print(f"\n📚 TAREAS DEL CAPÍTULO {chapter} ({len(tasks)})")
            for task in tasks:
                self.show_task_details(task)
        except ValueError:
            print("Debe ingresar un número de capítulo válido.")


# Función principal para ejecutar el agente
def main():
    print("🚀 Inicializando Agente IA para Gestión de Tareas del Curso...")
    agent = CourseTaskAgent()
    agent.interactive_menu()

if __name__ == "__main__":
    main()

🚀 Inicializando Agente IA para Gestión de Tareas del Curso...
No se encontró archivo de tareas. Creando uno nuevo...

🤖 AGENTE IA - GESTIÓN DE TAREAS DEL CURSO
1. Ver resumen de progreso
2. Ver todas las tareas
3. Ver tareas pendientes
4. Ver tareas vencidas
5. Buscar tareas
6. Actualizar estado de tarea
7. Sugerir próxima tarea
8. Generar plan de estudio
9. Ver tareas por capítulo
0. Salir
--------------------------------------------------
Seleccione una opción: 1

📊 RESUMEN DE PROGRESO
Total de tareas: 8
Completadas: 0 ✅
En progreso: 0 🔄
Pendientes: 8 ⏳
Vencidas: 0 ⚠️
Progreso: 0.0%
Horas estimadas totales: 49.5
Horas completadas: 0
Horas restantes: 49.5

Presione Enter para continuar...6

🤖 AGENTE IA - GESTIÓN DE TAREAS DEL CURSO
1. Ver resumen de progreso
2. Ver todas las tareas
3. Ver tareas pendientes
4. Ver tareas vencidas
5. Buscar tareas
6. Actualizar estado de tarea
7. Sugerir próxima tarea
8. Generar plan de estudio
9. Ver tareas por capítulo
0. Salir
-----------------------

**Punto 3**

In [4]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from datetime import datetime
from typing import List, Dict, Tuple

class SimpleRAG:
    """Sistema RAG simple y funcional para Universidad Nacional"""

    def __init__(self):
        print("🚀 Inicializando sistema RAG...")
        self.documents = []
        self.vectorizer = None
        self.vectors = None
        self.initialize_data()
        self.create_vectors()
        print("✅ Sistema inicializado correctamente")

    def initialize_data(self):
        """Inicializa con datos de la Universidad Nacional"""
        print("📚 Cargando información de la Universidad Nacional...")

        self.documents = [
            {
                'id': 1,
                'titulo': 'Universidad Nacional de Colombia - Información General',
                'contenido': '''La Universidad Nacional de Colombia es la universidad pública más importante de Colombia.
                Fue fundada el 22 de septiembre de 1867 por el General Santos Acosta. Su sede principal está ubicada
                en Bogotá en la Ciudad Universitaria. La universidad tiene 9 sedes en total: Bogotá, Medellín,
                Manizales, Palmira, Arauca, Leticia, San Andrés, Tumaco y La Paz. Es considerada la universidad
                más prestigiosa del país y una de las mejores de América Latina. Tiene más de 54000 estudiantes
                y 8000 profesores. Su lema es Trabajo y Rectitud. Ofrece programas de pregrado maestría y doctorado
                en todas las áreas del conocimiento.''',
                'categoria': 'general'
            },
            {
                'id': 2,
                'titulo': 'Proceso de Admisiones Universidad Nacional',
                'contenido': '''El proceso de admisión a la Universidad Nacional se realiza mediante el Examen de
                Admisión EAUN. Este examen evalúa cinco componentes: Análisis Textual, Matemáticas, Ciencias Naturales,
                Ciencias Sociales e Idioma extranjero. Se realizan dos convocatorias por año. Los aspirantes deben
                registrarse en línea, pagar derechos de participación y presentar documentos. El examen dura 4 horas
                y 30 minutos. Los resultados se publican un mes después. Para aplicar necesitas certificado de bachiller,
                documento de identidad, foto y comprobante de pago. El puntaje mínimo varía según el programa académico.''',
                'categoria': 'admisiones'
            },
            {
                'id': 3,
                'titulo': 'Sedes Universidad Nacional Colombia',
                'contenido': '''La Universidad Nacional tiene 9 sedes distribuidas por Colombia. Sede Bogotá es la
                más grande ubicada en Ciudad Universitaria con todas las facultades. Sede Medellín está en Robledo
                y es fuerte en ingenierías. Sede Manizales se especializa en ingeniería y ciencias agropecuarias.
                Sede Palmira en Valle del Cauca enfocada en ciencias agrarias. Sede Arauca tiene programas regionales.
                Sede Leticia ofrece programas amazónicos y ambientales. Sede San Andrés tiene programas marinos.
                Sede Tumaco enfocada en desarrollo del Pacífico. Sede La Paz con programas de desarrollo rural.''',
                'categoria': 'sedes'
            },
            {
                'id': 4,
                'titulo': 'Programas Académicos Universidad Nacional',
                'contenido': '''La Universidad Nacional ofrece más de 450 programas académicos. En pregrado tiene
                107 programas en 11 facultades: Artes, Ciencias, Ciencias Agrarias, Ciencias Económicas, Ciencias
                Humanas, Derecho, Enfermería, Ingeniería, Medicina, Medicina Veterinaria y Odontología. Las carreras
                más populares son Medicina, Ingeniería de Sistemas, Derecho, Psicología, Administración, Arquitectura,
                Biología, Matemáticas, Física y Química. También tiene más de 300 programas de posgrado incluyendo
                especializaciones, maestrías y doctorados en todas las áreas.''',
                'categoria': 'programas'
            },
            {
                'id': 5,
                'titulo': 'Investigación Universidad Nacional',
                'contenido': '''La Universidad Nacional es líder en investigación en Colombia. Tiene más de 500 grupos
                de investigación reconocidos por Minciencias. Investiga en biotecnología, nanotecnología, ciencias
                básicas, ingeniería, ciencias sociales, humanidades y salud. Publica más de 2000 artículos científicos
                anuales en revistas indexadas. Tiene convenios con universidades internacionales prestigiosas.
                Maneja patentes y transferencia tecnológica. Sus investigadores reciben reconocimientos nacionales
                e internacionales. Ofrece programas doctorales en múltiples áreas del conocimiento.''',
                'categoria': 'investigacion'
            },
            {
                'id': 6,
                'titulo': 'Facultades Universidad Nacional',
                'contenido': '''La Universidad Nacional tiene 11 facultades principales. Facultad de Ingeniería ofrece
                todas las ingenierías como sistemas, civil, industrial, mecánica, eléctrica, química. Facultad de
                Medicina incluye medicina, enfermería y otras ciencias de la salud. Facultad de Ciencias tiene
                matemáticas, física, química, biología, geología. Facultad de Ciencias Humanas incluye psicología,
                filosofía, historia, antropología. Facultad de Derecho ofrece derecho y ciencias políticas.
                Facultad de Artes tiene música, artes plásticas, diseño. También están Ciencias Económicas,
                Ciencias Agrarias, Odontología y Medicina Veterinaria.''',
                'categoria': 'facultades'
            },
            {
                'id': 7,
                'titulo': 'Servicios Estudiantiles Universidad Nacional',
                'contenido': '''La Universidad Nacional ofrece múltiples servicios para estudiantes. Bienestar
                Universitario incluye servicios médicos, psicológicos, deportivos y culturales. Los restaurantes
                universitarios ofrecen alimentación subsidiada. Hay residencias estudiantiles para estudiantes
                de otras ciudades. Sistema de becas y apoyos económicos para estudiantes de bajos recursos.
                Programas de acompañamiento académico y tutorías. Bibliotecas especializadas en cada sede.
                Servicios de salud con centros médicos. Actividades deportivas y recreativas. Grupos culturales
                y artísticos. Programas de intercambio nacional e internacional.''',
                'categoria': 'servicios'
            },
            {
                'id': 8,
                'titulo': 'Historia Universidad Nacional Colombia',
                'contenido': '''La Universidad Nacional de Colombia fue fundada en 1867 durante el gobierno del
                General Santos Acosta. Inicialmente se llamó Universidad Nacional de los Estados Unidos de Colombia.
                Su primer rector fue Ezequiel Rojas. Ha pasado por diferentes reformas y cambios a lo largo de
                su historia. Durante el siglo XX se expandió creando nuevas sedes y facultades. Ha sido protagonista
                de importantes movimientos estudiantiles y sociales. Sus egresados han ocupado cargos importantes
                en el gobierno, la academia y la sociedad colombiana. Es considerada la universidad más antigua
                y prestigiosa del país.''',
                'categoria': 'historia'
            }
        ]
        print(f"✅ Cargados {len(self.documents)} documentos")

    def create_vectors(self):
        """Crea los vectores TF-IDF"""
        print("🔧 Creando vectores...")

        try:
            # Extraer contenidos
            texts = [doc['contenido'].lower() for doc in self.documents]

            # Crear vectorizador
            self.vectorizer = TfidfVectorizer(
                max_features=1000,
                ngram_range=(1, 2),
                min_df=1,
                stop_words=None
            )

            # Crear vectores
            self.vectors = self.vectorizer.fit_transform(texts)
            print(f"✅ Vectores creados: {self.vectors.shape}")

        except Exception as e:
            print(f"❌ Error creando vectores: {e}")
            # Inicialización básica como respaldo
            self.vectorizer = TfidfVectorizer()
            self.vectors = self.vectorizer.fit_transform([doc['contenido'] for doc in self.documents])

    def search(self, query: str, top_k: int = 3) -> List[Tuple[Dict, float]]:
        """Busca documentos relevantes"""
        if not query.strip():
            return []

        try:
            # Procesar consulta
            query_processed = query.lower()
            query_vector = self.vectorizer.transform([query_processed])

            # Calcular similitudes
            similarities = cosine_similarity(query_vector, self.vectors).flatten()

            # Obtener mejores resultados
            top_indices = np.argsort(similarities)[::-1][:top_k]

            results = []
            for idx in top_indices:
                score = similarities[idx]
                if score > 0.01:  # Umbral muy bajo
                    results.append((self.documents[idx], score))

            return results

        except Exception as e:
            print(f"❌ Error en búsqueda: {e}")
            return []

    def answer(self, question: str) -> str:
        """Responde una pregunta"""
        print(f"\n🔍 Buscando: '{question}'")

        # Buscar documentos relevantes
        results = self.search(question, top_k=3)

        if not results:
            return self.get_help_message()

        # Construir respuesta
        response = "📋 **Respuesta:**\n\n"

        # Mostrar el documento más relevante completo
        best_doc, best_score = results[0]
        response += f"**{best_doc['titulo']}**\n\n"
        response += f"{best_doc['contenido']}\n\n"

        # Si hay más resultados relevantes, mostrar resumen
        if len(results) > 1:
            response += "📚 **Información adicional encontrada:**\n"
            for doc, score in results[1:]:
                response += f"• {doc['titulo']} (relevancia: {score:.2f})\n"

        return response

    def get_help_message(self) -> str:
        """Mensaje de ayuda con ejemplos"""
        return """❌ No encontré información específica para tu consulta.

🤖 **Prueba con estas preguntas:**

📍 **Información General:**
• ¿Qué es la Universidad Nacional de Colombia?
• ¿Cuándo fue fundada la Universidad Nacional?
• ¿Cuántas sedes tiene la universidad?

🎓 **Admisiones:**
• ¿Cómo es el proceso de admisión?
• ¿Qué examen debo presentar?
• ¿Qué documentos necesito para aplicar?

🏫 **Programas y Facultades:**
• ¿Qué carreras puedo estudiar?
• ¿Cuáles son las facultades?
• ¿Hay programas de ingeniería?

🔬 **Investigación:**
• ¿Cómo es la investigación en la universidad?
• ¿Hay programas de doctorado?

🏢 **Sedes:**
• ¿Dónde están ubicadas las sedes?
• ¿Qué hay en la sede de Bogotá?

💡 **Tip:** Usa palabras clave como "admisión", "carreras", "sedes", etc."""

    def list_categories(self):
        """Lista las categorías disponibles"""
        categories = set(doc['categoria'] for doc in self.documents)
        print("\n📂 **Categorías disponibles:**")
        for cat in sorted(categories):
            count = sum(1 for doc in self.documents if doc['categoria'] == cat)
            print(f"• {cat.title()}: {count} documento(s)")

    def search_by_category(self, category: str):
        """Busca por categoría"""
        docs = [doc for doc in self.documents if doc['categoria'].lower() == category.lower()]
        if docs:
            print(f"\n📑 **Documentos en '{category}':**")
            for doc in docs:
                print(f"• {doc['titulo']}")
        else:
            print(f"❌ No se encontraron documentos en la categoría '{category}'")

def main():
    """Función principal"""
    print("=" * 60)
    print("🎓 CHAT UNIVERSIDAD NACIONAL DE COLOMBIA")
    print("=" * 60)

    # Inicializar sistema
    rag = SimpleRAG()

    print("\n🎯 **El sistema está listo para responder tus preguntas**")
    print("\n🤖 **Comandos especiales:**")
    print("• 'categorias' - Ver categorías de información")
    print("• 'categoria <nombre>' - Ver documentos por categoría")
    print("• 'ayuda' - Ver ejemplos de preguntas")
    print("• 'salir' - Terminar")

    print("\n" + "─" * 60)

    while True:
        try:
            pregunta = input("\n💬 Tu pregunta: ").strip()

            if not pregunta:
                continue

            # Comandos especiales
            if pregunta.lower() in ['salir', 'exit', 'quit']:
                print("\n👋 ¡Hasta luego!")
                break
            elif pregunta.lower() == 'ayuda':
                print(rag.get_help_message())
                continue
            elif pregunta.lower() == 'categorias':
                rag.list_categories()
                continue
            elif pregunta.lower().startswith('categoria '):
                cat = pregunta[10:].strip()
                rag.search_by_category(cat)
                continue

            # Responder pregunta
            respuesta = rag.answer(pregunta)
            print(f"\n{respuesta}")

        except KeyboardInterrupt:
            print("\n\n👋 ¡Hasta luego!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")

if __name__ == "__main__":
    main()

🎓 CHAT UNIVERSIDAD NACIONAL DE COLOMBIA
🚀 Inicializando sistema RAG...
📚 Cargando información de la Universidad Nacional...
✅ Cargados 8 documentos
🔧 Creando vectores...
✅ Vectores creados: (8, 808)
✅ Sistema inicializado correctamente

🎯 **El sistema está listo para responder tus preguntas**

🤖 **Comandos especiales:**
• 'categorias' - Ver categorías de información
• 'categoria <nombre>' - Ver documentos por categoría
• 'ayuda' - Ver ejemplos de preguntas
• 'salir' - Terminar

────────────────────────────────────────────────────────────

💬 Tu pregunta: ¿Qué es la Universidad Nacional?

🔍 Buscando: '¿Qué es la Universidad Nacional?'

📋 **Respuesta:**

**Universidad Nacional de Colombia - Información General**

La Universidad Nacional de Colombia es la universidad pública más importante de Colombia. 
                Fue fundada el 22 de septiembre de 1867 por el General Santos Acosta. Su sede principal está ubicada 
                en Bogotá en la Ciudad Universitaria. La universidad ti